## The following notebook contains examples of analysing A Walmart stock CSV obtained from Kaggle.

#### Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("exercises").getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [6]:
df = spark.read.csv("walmart_stock.csv",header=True,
                 inferSchema=True)

#### What are the column names?

In [24]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [20]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [115]:
#df.select(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']).show(5)
for row in df.head(5):
    print (row)
    print ('\n')

Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




#### Use describe() to learn about the DataFrame.

In [36]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [43]:
from pyspark.sql.functions import format_number


#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [49]:
df1 = df.withColumn('HV Ratio',df['High']/df['Volume']).show()
#df1 = df.withColumn('HV Ratio',df['High']/df['Volume']).select('HV Ratio').show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|            HV Ratio|
+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09|         59.029999|   

#### What day had the Peak High in Price?

In [59]:
df.orderBy(df['High'].desc()).select(['Date','High']).show()

+----------+-----------------+
|      Date|             High|
+----------+-----------------+
|2015-01-13|        90.970001|
|2015-01-08|90.66999799999999|
|2015-01-09|        90.389999|
|2015-01-12|        90.309998|
|2015-01-23|        89.260002|
|2015-01-26|        89.160004|
|2015-01-07|            88.68|
|2015-01-14|        88.519997|
|2015-01-27|        88.459999|
|2015-01-22|        88.400002|
|2015-01-28|        88.230003|
|2014-11-28|        88.089996|
|2015-02-06|             88.0|
|2015-01-15|        87.779999|
|2015-01-29|        87.720001|
|2015-01-20|        87.699997|
|2015-01-16|        87.459999|
|2014-12-31|        87.440002|
|2015-02-10|        87.410004|
|2015-01-30|        87.360001|
+----------+-----------------+
only showing top 20 rows



#### What is the mean of the Close column?

In [67]:
from pyspark.sql.functions import avg, max, min
df.select(avg('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [71]:
df.select(max('Volume')).show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [72]:
df.select(min('Volume')).show()

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



#### How many days was the Close lower than 60 dollars?

In [74]:
df.filter('Close<60').count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [79]:
data1 = df.filter('High>80').count()

In [80]:
from pyspark.sql.functions import countDistinct
data2 = df.select(countDistinct("Date")).show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                1258|
+--------------------+



#### What is the Pearson correlation between High and Volume?

from pyspark.sql.functions import corr

In [88]:
df.select(corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [95]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
#df.select(dayofmonth(df['Date'])).show()
df.groupBy('Date').max().show()

+----------+------------------+---------+------------------+------------------+-----------+-----------------+
|      Date|         max(Open)|max(High)|          max(Low)|        max(Close)|max(Volume)|   max(Adj Close)|
+----------+------------------+---------+------------------+------------------+-----------+-----------------+
|2012-04-17|61.110001000000004|61.950001|             61.09|         61.869999|   10676400|54.32359399999999|
|2013-01-22|         69.050003|69.650002|         68.959999| 69.58000200000001|    5747900|62.18356899999999|
|2013-03-26|             74.93|75.089996|             74.43|         74.769997|    6642400|67.25296999999999|
|2013-05-21|         77.620003|    78.18|             77.25|         77.389999|    7484300|        70.027089|
|2013-09-09|         72.849998|73.650002|         72.699997|         73.510002|    5523000|66.92014300000001|
|2014-09-26|             76.25|    76.57|         75.860001|         76.489998|    3752900|        71.393879|
|2014-11-1

In [112]:
df.groupBy(year('Date').alias('Year')).max().select('Year','max(High)').orderBy('Year').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [111]:
df.groupBy(month('Date').alias('Month')).avg().select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

